In [1]:
import meteostat
import pandas as pd
from pyncei import NCEIBot
import matplotlib.pyplot as plt 
import time
import requests

import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point

from lib import data_loaders

import os
from dotenv import load_dotenv, dotenv_values 
# loading variables from .env file
load_dotenv() 

#os.getenv("NCEI_KEY")
ncei = NCEIBot(os.getenv("NCEI_KEY"))

In [ ]:
dp_poly = gpd.read_file("./gis/tnc.geojson")
# dangermond is in utm zone 10n, i.e. EPSG:32610
dp_poly = dp_poly.to_crs(epsg="32610")
# create buffer
dp_buffer = dp_poly.geometry.buffer(30000) # 30km

In [ ]:
inventory_df = pd.read_table(
    "https://www.ncei.noaa.gov/pub/data/ghcn/daily/ghcnd-inventory.txt", 
    sep='\s+', 
    names=["id", "lat", "lon", "var", "start", "end"]
)

In [ ]:
# need data from 1979-2023

# start by fetching inventory data from NCEI GHCNd
# cols described here: https://www.ncei.noaa.gov/products/land-based-station/global-historical-climatology-network-daily
inventory_df = pd.read_table(
    "https://www.ncei.noaa.gov/pub/data/ghcn/daily/ghcnd-inventory.txt", 
    sep='\s+', 
    names=["id", "lat", "lon", "var", "start", "end"]
)

# also download "stations" directory, in order to add station names to the inventory
ncei_stations = pd.read_fwf(
    "https://www.ncei.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt", 
    header=None,
    sep='\s+', 
)

# station name cols, which are not parsed correctly by pandas (combine 4,5)
ncei_stations['station_name_full'] = ncei_stations[
    ncei_stations[4].notna()][4] + ncei_stations[ncei_stations[5].notna()
][5]

# collapse back to single col
ncei_stations['station_name_full'] = ncei_stations['station_name_full'].fillna(ncei_stations[4])
ncei_stations = ncei_stations.drop([4,5], axis=1)
ncei_stations.columns = ["id", "lat", "lon", "elev", "network", "num", "station_name_full"]

# combine on id
#inventory_df = pd.concat([inventory_df, ncei_stations], keys='id')

In [ ]:
# create GeoDataFrame
geometry = [Point(xy) for xy in zip(inventory_df.lon, inventory_df.lat)]
df = inventory_df.drop(['lon', 'lat'], axis=1)
gdf = GeoDataFrame(df, crs="EPSG:4326", geometry=geometry)

# dangermond is in utm zone 10n, i.e. EPSG:32610
gdf = gdf.to_crs(epsg="32610")


In [ ]:
mask = gdf.within(dp_buffer.iloc[0])
gdf_near = gdf.loc[mask]

In [ ]:
gdf_near

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))

gdf_near.plot(ax=ax)
dp_poly.plot(ax=ax)

In [ ]:
gdf_near['id'].unique()

In [ ]:
# check period of record for nearby sites before collecting:
for site in gdf_near['id'].unique():
    print(site)
    start = inventory_df[inventory_df.id == site]['start'].min()
    end = inventory_df[inventory_df.id == site]['end'].max()

    #if end > 1978: # only want data during study period
    print(f"start: {start}, end: {end}")

    print("---")

In [ ]:
ghcnd_data = {}

# get data for nearby stations
for site in gdf_near['id'].unique():
    print(site)
    start = inventory_df[inventory_df.id == site]['start'].min()
    end = inventory_df[inventory_df.id == site]['end'].max()

    if end > 1978: # only want stations with data during study period

        print(f"start: {start}, end: {end}")
        
        if start < 1978: # don't request years before start of site
            start = 1978

        years = []

        for year in range(start-1, end+1):
            print(f"requesting data for {site}")
            print(year)
            # NCEI data access for all networks
            response = ncei.get_data(
                datasetid="GHCND",
                stationid=[f"GHCND:{site}"], 
                #datatypeid=["TMIN", "TMAX"], # all for now
                startdate=f"{year}-01-01",
                enddate=f"{year}-12-31",
            )

            df_year = response.to_dataframe()
            
            if not df_year.empty:
                years.append(df_year)

            time.sleep(6) # slow down requests a bit

        df_site = pd.concat(years)
        ghcnd_data[site] = df_site

    print("---")

In [ ]:
ghcnd_data.keys()

In [ ]:
for stn in ghcnd_data.keys():
    df = ghcnd_data[stn]
    start = df['date'].min().strftime("%Y%m%d")
    end = df['date'].max().strftime("%Y%m%d")
    #df.to_parquet(f"./output/{stn}_{start}_{end}.parquet")

In [ ]:
# Get HADS data from Iowa Environmental Mesonet
df_ca_awos = pd.read_csv("https://mesonet.agron.iastate.edu/sites/networks.php?network=CA_ASOS&format=csv&nohtml=on")
df_ca_coop = pd.read_csv("https://mesonet.agron.iastate.edu/sites/networks.php?network=CA_COOP&format=csv&nohtml=on")
df_ca_dcp = pd.read_csv("https://mesonet.agron.iastate.edu/sites/networks.php?network=CA_DCP&format=csv&nohtml=on")

ca_iem = pd.concat([df_ca_awos, df_ca_coop, df_ca_dcp])

# create GeoDataFrame
geometry = [Point(xy) for xy in zip(ca_iem.lon, ca_iem.lat)]
df = ca_iem.drop(['lon', 'lat'], axis=1)
iem_gdf = GeoDataFrame(df, crs="EPSG:4326", geometry=geometry)

# dangermond is in utm zone 10n, i.e. EPSG:32610
iem_gdf = iem_gdf.to_crs(epsg="32610")

iem_mask = iem_gdf.within(dp_buffer.iloc[0])
iem_gdf_near = iem_gdf.loc[iem_mask]

In [ ]:
iem_gdf_near